In [3]:
#from generator import UNETGenerator
from keras.layers import Activation, Input, Dropout, merge,concatenate
from keras.layers.convolutional import Convolution2D, UpSampling2D
from keras.layers import Conv2D,Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model

Using TensorFlow backend.


In [4]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [5]:
# ---------------------------------------------
# HYPER PARAMS
# ---------------------------------------------
# width, height of images to work with. Assumes images are square
im_width = im_height = 256

# inpu/oputputt channels in image
input_channels = 3
output_channels = 3

# image dims
input_img_dim = (im_width, im_height,input_channels)
output_img_dim = (im_width, im_height,output_channels)

In [6]:
def UNETGenerator(input_img_dim, num_output_channels):
    """
    Creates the generator according to the specs in the paper below.
    It's basically a skip layer AutoEncoder
    Generator does the following:
    1. Takes in an image
    2. Generates an image from this image
    Differs from a standard GAN because the image isn't random.
    This model tries to learn a mapping from a suboptimal image to an optimal image.
    [https://arxiv.org/pdf/1611.07004v1.pdf][5. Appendix]
    :param input_img_dim: (channel, height, width)
    :param output_img_dim: (channel, height, width)
    :return:
    """
    # -------------------------------
    # ENCODER
    # C64-C128-C256-C512-C512-C512-C512-C512
    # 1 layer block = Conv - BN - LeakyRelu
    # -------------------------------
    stride = 2
    merge_mode = 'concat'

    # batch norm mode
    bn_mode = 2

    # batch norm merge axis
    bn_axis = 3
    bn_axis_1 = 3

    input_layer = Input(shape=input_img_dim, name="unet_input")

    # 1 encoder C64
    # skip batchnorm on this layer on purpose (from paper)
    en_1 = Convolution2D(nb_filter=64, nb_row=4, nb_col=4, border_mode='same', subsample=(stride, stride))(input_layer)
    #en_1 = Conv2D(filters=64,kernel_size=4,padding="same",strides=2)(input_layer)
    en_1 = LeakyReLU(alpha=0.2)(en_1)

#     # 2 encoder C128
    en_2 = Convolution2D(nb_filter=64, nb_row=4, nb_col=4, border_mode='same', subsample=(stride, stride))(en_1)
    #en_2 = Conv2D(filters=64,kernel_size=4,padding="same",strides=2)(en_1)
    en_2 = BatchNormalization(name='gen_en_bn_2', axis=bn_axis)(en_2)
    en_2 = LeakyReLU(alpha=0.2)(en_2)

    # 3 encoder C256
    en_3 = Convolution2D(nb_filter=256, nb_row=4, nb_col=4, border_mode='same', subsample=(stride, stride))(en_2)
    en_3 = BatchNormalization(name='gen_en_bn_3', axis=bn_axis)(en_3)
    en_3 = LeakyReLU(alpha=0.2)(en_3)

    # 4 encoder C512
    en_4 = Convolution2D(nb_filter=512, nb_row=4, nb_col=4, border_mode='same', subsample=(stride, stride))(en_3)
    en_4 = BatchNormalization(name='gen_en_bn_4', axis=bn_axis)(en_4)
    en_4 = LeakyReLU(alpha=0.2)(en_4)

    # 5 encoder C512
    en_5 = Convolution2D(nb_filter=512, nb_row=4, nb_col=4, border_mode='same', subsample=(stride, stride))(en_4)
    en_5 = BatchNormalization(name='gen_en_bn_5', axis=bn_axis)(en_5)
    en_5 = LeakyReLU(alpha=0.2)(en_5)

    # 6 encoder C512
    en_6 = Convolution2D(nb_filter=512, nb_row=4, nb_col=4, border_mode='same', subsample=(stride, stride))(en_5)
    en_6 = BatchNormalization(name='gen_en_bn_6', axis=bn_axis)(en_6)
    en_6 = LeakyReLU(alpha=0.2)(en_6)

    # 7 encoder C512
    en_7 = Convolution2D(nb_filter=512, nb_row=4, nb_col=4, border_mode='same', subsample=(stride, stride))(en_6)
    en_7 = BatchNormalization(name='gen_en_bn_7', axis=bn_axis)(en_7)
    en_7 = LeakyReLU(alpha=0.2)(en_7)

    # 8 encoder C512
    en_8 = Convolution2D(nb_filter=512, nb_row=4, nb_col=4, border_mode='same', subsample=(stride, stride))(en_7)
    en_8 = BatchNormalization(name='gen_en_bn_8', axis=bn_axis)(en_8)
    en_8 = LeakyReLU(alpha=0.2)(en_8)
    
#    unet_generator = Model(input=[input_layer], output=[en_8], name='unet_generator')


    # -------------------------------
    # DECODER
    # CD512-CD1024-CD1024-C1024-C1024-C512-C256-C128
    # 1 layer block = Conv - Upsample - BN - DO - Relu
    # also adds skip connections (merge). Takes input from previous layer matching encoder layer
    # -------------------------------
    # 1 decoder CD512 (decodes en_8)
    de_1 = UpSampling2D(size=(2, 2))(en_8)
    de_1 = Convolution2D(nb_filter=512, nb_row=4, nb_col=4, border_mode='same')(de_1)
    de_1 = BatchNormalization(name='gen_de_bn_1', axis=bn_axis)(de_1)
    de_1 = Dropout(p=0.5)(de_1)
    #de_1 = merge([de_1, en_7], mode=merge_mode, concat_axis=1)
    de_1 = concatenate([de_1, en_7],axis = bn_axis_1)
    de_1 = Activation('relu')(de_1)

    # 2 decoder CD1024 (decodes en_7)
    de_2 = UpSampling2D(size=(2, 2))(de_1)
    de_2 = Convolution2D(nb_filter=1024, nb_row=4, nb_col=4, border_mode='same')(de_2)
    de_2 = BatchNormalization(name='gen_de_bn_2', axis=bn_axis)(de_2)
    de_2 = Dropout(p=0.5)(de_2)
    #de_2 = merge([de_2, en_6], mode=merge_mode, concat_axis=1)
    de_2 = concatenate([de_2, en_6],axis = bn_axis_1)
    de_2 = Activation('relu')(de_2)

    # 3 decoder CD1024 (decodes en_6)
    de_3 = UpSampling2D(size=(2, 2))(de_2)
    de_3 = Convolution2D(nb_filter=1024, nb_row=4, nb_col=4, border_mode='same')(de_3)
    de_3 = BatchNormalization(name='gen_de_bn_3', axis=bn_axis)(de_3)
    de_3 = Dropout(p=0.5)(de_3)
    #de_3 = merge([de_3, en_5], mode=merge_mode, concat_axis=1)
    de_3 = concatenate([de_3, en_5],axis = bn_axis_1)
    de_3 = Activation('relu')(de_3)

    # 4 decoder CD1024 (decodes en_5)
    de_4 = UpSampling2D(size=(2, 2))(de_3)
    de_4 = Convolution2D(nb_filter=1024, nb_row=4, nb_col=4, border_mode='same')(de_4)
    de_4 = BatchNormalization(name='gen_de_bn_4', axis=bn_axis)(de_4)
    de_4 = Dropout(p=0.5)(de_4)
    #de_4 = merge([de_4, en_4], mode=merge_mode, concat_axis=1)
    de_4 = concatenate([de_4, en_4],axis = bn_axis_1)
    de_4 = Activation('relu')(de_4)

    # 5 decoder CD1024 (decodes en_4)
    de_5 = UpSampling2D(size=(2, 2))(de_4)
    de_5 = Convolution2D(nb_filter=1024, nb_row=4, nb_col=4, border_mode='same')(de_5)
    de_5 = BatchNormalization(name='gen_de_bn_5', axis=bn_axis)(de_5)
    de_5 = Dropout(p=0.5)(de_5)
    #de_5 = merge([de_5, en_3], mode=merge_mode, concat_axis=1)
    de_5 = concatenate([de_5, en_3],axis = bn_axis_1)
    de_5 = Activation('relu')(de_5)

    # 6 decoder C512 (decodes en_3)
    de_6 = UpSampling2D(size=(2, 2))(de_5)
    de_6 = Convolution2D(nb_filter=512, nb_row=4, nb_col=4, border_mode='same')(de_6)
    de_6 = BatchNormalization(name='gen_de_bn_6', axis=bn_axis)(de_6)
    de_6 = Dropout(p=0.5)(de_6)
    #de_6 = merge([de_6, en_2], mode=merge_mode, concat_axis=1)
    de_6 = concatenate([de_6, en_2],axis = bn_axis_1)
    de_6 = Activation('relu')(de_6)

    # 7 decoder CD256 (decodes en_2)
    de_7 = UpSampling2D(size=(2, 2))(de_6)
    de_7 = Convolution2D(nb_filter=256, nb_row=4, nb_col=4, border_mode='same')(de_7)
    de_7 = BatchNormalization(name='gen_de_bn_7', axis=bn_axis)(de_7)
    de_7 = Dropout(p=0.5)(de_7)
    #de_7 = merge([de_7, en_1], mode=merge_mode, concat_axis=1)
    de_7 = concatenate([de_7, en_1],axis = bn_axis_1)
    de_7 = Activation('relu')(de_7)

    # After the last layer in the decoder, a convolution is applied
    # to map to the number of output channels (3 in general,
    # except in colorization, where it is 2), followed by a Tanh
    # function.
    de_8 = UpSampling2D(size=(2, 2))(de_7)
    de_8 = Convolution2D(nb_filter=num_output_channels, nb_row=4, nb_col=4, border_mode='same')(de_8)
    # 最后一层在部分得layer得左上部分数值过大，导致拟合效果不好，加一个batchnorm试试
    #de_8 = BatchNormalization(name='gen_de_bn_8', axis=bn_axis)(de_8)
    de_8 = Activation('tanh')(de_8)

    unet_generator = Model(input=[input_layer], output=[de_8], name='unet_generator')
    return unet_generator


In [7]:
generator_nn = UNETGenerator(input_img_dim=input_img_dim, num_output_channels=output_channels)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(4, 4), filters=64, strides=(2, 2), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(4, 4), filters=64, strides=(2, 2), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(4, 4), filters=256, strides=(2, 2), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(4, 4), filters=512, strides=(2, 2), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(4, 4), filters=512, strides=(2, 2), padding="same")`
/usr/local/l

In [8]:
generator_nn.summary()

Model: "unet_generator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
unet_input (InputLayer)         (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 64) 3136        unet_input[0][0]                 
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 128, 128, 64) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 64, 64, 64)   65600       leaky_re_lu_1[0][0]              
_____________________________________________________________________________________

In [9]:
import cv2
my_pic = cv2.imread("./facades/test/1.jpg")
my_pic=  my_pic[:,256:,:]
my_pic = my_pic.reshape(1,256,256,3)
my_pic = (my_pic - 127.5) / 127.5

In [10]:
import matplotlib.pyplot as plt
def plot_weight(model,layer,my_pic):
    print(generator_nn.layers[layer].name)
    intermediate_layer_model = Model(inputs=model.input
                                     ,outputs=model.layers[layer].output)
    fake_pic = intermediate_layer_model.predict(my_pic)
    fake_pic = fake_pic * 127.5 + 127.5
    fake_pic = fake_pic.astype(np.uint8)
    print(fake_pic.shape)
    num = fake_pic.shape[-1]
    plt.figure()
    for i in range(1,min(25,num+1)):
        #print(i)
        plt.subplot(5,5,i)
        plt.imshow(fake_pic[0][...,i-1])

    #     plt.xticks([])
    #     plt.yticks([])
    plt.show()

In [11]:
def get_weight(model,layer,my_pic):
    print(generator_nn.layers[layer].name,layer)
    intermediate_layer_model = Model(inputs=model.input
                                     ,outputs=model.layers[layer].output)
    fake_pic = intermediate_layer_model.predict(my_pic)
    #print(fake_pic.shape)
    print(fake_pic[0,:10,:10,:].mean(0).mean(0)[:30])
    print(fake_pic[0,-10:,-10:,:].mean(0).mean(0)[:30])
    #print(fake_pic[0].mean(2))
    print("*"*20)
    return fake_pic

In [12]:
from keras.optimizers import Adam
opt_discriminator = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#这把loss设置为mse加大对异常值的敏感度
generator_nn.compile(loss='mae', optimizer=opt_discriminator)

In [13]:
import pickle as pickle
import random
import os
import numpy as np
ban_num = 2

In [14]:
import cv2
my_pic = cv2.imread("./facades/train/28.jpg")
my_pic = (my_pic - 127.5)/127.5
my_pic_b=  my_pic[:,256:,:]
my_pic_a=  my_pic[:,:256,:]
my_pic_a = my_pic_a.reshape(1,256,256,3)
my_pic_b = my_pic_b.reshape(1,256,256,3)

In [15]:
bad_case = [1, 2, 4, 5, 6, 11, 12, 14, 22, 24, 25, 29, 35, 38, 39, 43, 44, 45, 47, 49, 51, 52, 54, 55, 56, 59, 63, 64, 76, 79, 82, 87, 92, 96, 98, 100, 102, 103, 104, 106, 108, 109, 110, 111, 115, 116, 117, 118, 119, 120, 123, 124, 125, 126, 127, 129, 132, 133, 134, 136, 141, 142, 143, 145, 149, 150, 151, 153, 154, 155, 156, 157, 158, 159, 160, 162, 163, 164, 165, 166, 167, 168, 169, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 187, 189, 190, 191, 193, 194, 195, 196, 197, 198, 199, 200, 209, 214, 216, 217, 218, 223, 224, 225, 226, 230, 233, 237, 238, 241, 244, 246, 249, 253, 258, 259, 260, 261, 262, 264, 267, 268, 269, 273, 278, 279, 280, 281, 287, 292, 294, 295]

In [16]:
!mkdir pic_fuck

mkdir: cannot create directory ‘pic_fuck’: File exists


In [18]:
disc_loss =0
epoch=0
for j in range(100,300):
#for i in bad_case: 
    i = 124
    disc_loss_cur =  generator_nn.train_on_batch(my_pic_b,my_pic_a)
    disc_loss += disc_loss_cur
    output_pic =  generator_nn.predict(my_pic_b)
    fake_pic = generator_nn.predict(my_pic_b)
    fake_pic = np.concatenate((fake_pic,output_pic,my_pic_a),1)
    fake_pic = fake_pic * 127.5 + 127.5
    cv2.imwrite("./pic_fuck/result_{}.jpg".format(j),fake_pic[0])
    print("epoch:{} {} loss:{}".format(epoch,j,disc_loss_cur))

epoch:0 100 loss:0.17090296745300293
epoch:0 101 loss:0.1711486577987671
epoch:0 102 loss:0.17033830285072327
epoch:0 103 loss:0.1691049039363861
epoch:0 104 loss:0.16838137805461884
epoch:0 105 loss:0.16884303092956543
epoch:0 106 loss:0.1692100614309311
epoch:0 107 loss:0.16828802227973938
epoch:0 108 loss:0.16637739539146423
epoch:0 109 loss:0.1677776575088501
epoch:0 110 loss:0.16501496732234955
epoch:0 111 loss:0.16474352777004242
epoch:0 112 loss:0.16503986716270447
epoch:0 113 loss:0.165230393409729
epoch:0 114 loss:0.1638486385345459
epoch:0 115 loss:0.16370289027690887
epoch:0 116 loss:0.16269466280937195
epoch:0 117 loss:0.1619585156440735
epoch:0 118 loss:0.1613035500049591
epoch:0 119 loss:0.16316251456737518
epoch:0 120 loss:0.16128239035606384
epoch:0 121 loss:0.15965795516967773
epoch:0 122 loss:0.16200003027915955
epoch:0 123 loss:0.159181147813797
epoch:0 124 loss:0.1601162552833557
epoch:0 125 loss:0.16024552285671234
epoch:0 126 loss:0.15847468376159668
epoch:0 127 l

In [2]:
for j in range(0,100):
#for i in bad_case: 
    i = 124
    disc_loss_cur =  generator_nn.train_on_batch(true_iamge[i:i+1],origin_image[i:i+1])
    disc_loss += disc_loss_cur
    output_pic =  generator_nn.predict(true_iamge[i:i+1])
    fake_pic = generator_nn.predict(my_pic)
    fake_pic = np.concatenate((fake_pic,output_pic,origin_image[i:i+1]),1)
    fake_pic = fake_pic * 127.5 + 127.5
    cv2.imwrite("./pic_fuck_191/result_{}.jpg".format(j),fake_pic[0])
    print("epoch:{} {} loss:{}".format(epoch,j,disc_loss_cur))

NameError: name 'generator_nn' is not defined

In [19]:
import cv2
my_pic = cv2.imread("./facades/test/4.jpg")
my_pic=  my_pic[:,256:,:]
my_pic = my_pic.reshape(1,256,256,3)
my_pic = (my_pic - 127.5) / 127.5
fake_pic = generator_nn.predict(my_pic)
fake_pic = fake_pic * 127.5 + 127.5
cv2.imwrite("result.jpg".format(epoch),fake_pic[0])

True

In [46]:
import cv2
my_pic = cv2.imread("./facades/test/49.jpg")

In [48]:
HSV = cv2.cvtColor(my_pic, cv2.COLOR_BGR2HSV)